In [5]:
import pandas as pd
import numpy as np
import altair as alt

# 1. leggi il file
df = pd.read_json("src/assets/Teildatensatz.json")

# 2. tieni solo Bahnhofstrasse Nord
df_loc = df[df["location_id"] == 331].copy()

# 3. crea colonna date (solo giorno) e colonna ora
df_loc["date"] = pd.to_datetime(df_loc[["year", "month", "day"]])
df_loc["hour"] = pd.to_datetime(df_loc["timestamp"]).dt.hour

# 4. scegli il giorno che ti interessa
target_date = "2024 07 15"
df_day = df_loc[df_loc["date"] == target_date].copy()

# 5. differenza e vincitore
df_day["delta"] = (
    df_day["adult_ltr_pedestrians_count"]
    - df_day["adult_rtl_pedestrians_count"]
)

df_day["winner"] = np.where(
    df_day["delta"] > 0,
    "LTR > RTL",
    "RTL ≥ LTR",
)


In [36]:
chart = (
    alt.Chart(df_day)
    .mark_bar()
    .encode(
        x=alt.X(
            "hour:O",
            title="Stunde des Tages"
        ),
        y=alt.Y(
            "delta:Q",
            title="Differenz LTR minus RTL"
        ),
        color=alt.Color(
            "winner:N",
            scale=alt.Scale(range=["#4f8f4e", "#c0392b"]),
            legend=alt.Legend(
                orient="none",
                legendX=380,
                legendY=-40,
                direction="horizontal",
                title="Richtung",
                titleAnchor='middle'
            )
        ),
        tooltip=[
            alt.Tooltip("hour:O", title="Stunde"),
            alt.Tooltip("delta:Q", title="Differenz"),
            alt.Tooltip("winner:N", title="Richtung"),
        ],
    )
    .properties(
        width=900,
        height=500,
    )
)
chart


alt.Chart(...)

In [37]:
import json
spec = chart.to_dict()

with open("src/json_spec/fokusfrage.json", "w") as f:
    json.dump(spec, f, indent=2)